In [1]:
import glob
import pickle
import numpy as np
import pandas as pd
from library.preprocessing import new_Processing_before_journal

# Getting Folders

In [2]:
all_folders=glob.glob("./Trails/*/*/")

test_dates=['2019-06-28','2019-06-30','2019-07-01','2019-07-02','2019-07-03','2019-07-04','2019-07-05']
formatted_test_dates=['-'.join(d.split("-")[::-1]) for d in test_dates] #reversing order

#train folders
train_folders=[f for f in all_folders if f.split("\\")[-2].split("_")[0] not in formatted_test_dates]

#test folders
test_folders=[f for f in all_folders if f.split("\\")[-2].split("_")[0] in formatted_test_dates]

# Loading Models

In [3]:
with open('./logs/rfs.pickle', 'rb') as handle:
    rfs = pickle.load(handle)

# Features

In [4]:
#feature # binding
F=\
    {1:'stay_duration', #f1
     2:'mfcc0',3:'mfcc1',4:'mfcc2',5:'mfcc3',6:'mfcc4', #f2,f3,f4,f5,f6
     7:'wifi_count', 8:'edge_wifi_count', #f7,f8
     9:'RSI', #f9
     10:'human_made', 11:'natural_land',12:'road_exist_percent',13:'highly_populated_poi_exist'#f10,f11,f12,f13
    }

selected_feat=\
    {
        'Is_Bus_stop':[F[e] for e in [10,12,11,1,8]],
        'Is_Turn':[F[e] for e in [10,12,11,1,8,9]],
        'Is_Signal':[F[e] for e in [12,11,10,8,9]],
        'Is_Congestion':[F[e] for e in [2,11,10,9,12,8,6,1]],
        'Is_Adhoc':[F[e] for e in [10,11,1,12,2]]
    }

# Processing Training DATA

In [5]:
def process_training_data(folder):
    we_need=['time_zone','start_date','start_time','lat','long','stay_duration']+list(rfs.keys())
    return new_Processing_before_journal(folder+"ALL_DATA.csv")[we_need]

In [6]:
for folder in train_folders:
    f_df=process_training_data(folder)
    formatted_date='-'.join(folder.split("\\")[-2].split("_")[0].split("-")[::-1])
    f_df.start_date=formatted_date
    f_df.to_csv(folder+f"mu_serv_{formatted_date}.csv",index=False)

# Processing Testing DATA

In [7]:
def process_testing_data(folder):
    we_need=['time_zone','start_date','start_time','lat','long','stay_duration']+list(rfs.keys())
    df= new_Processing_before_journal(folder+"ALL_DATA.csv")
    for poi_column in rfs.keys():
        feature_names=selected_feat[poi_column]
        test_data = df[feature_names].values
        pred=rfs[poi_column].predict(test_data)
        df[poi_column]=pred
    return df[we_need]

In [8]:
for folder in test_folders:
    f_df=process_testing_data(folder)
    formatted_date='-'.join(folder.split("\\")[-2].split("_")[0].split("-")[::-1])
    f_df.start_date=formatted_date
    f_df.to_csv(folder+f"mu_serv_{formatted_date}.csv",index=False)

In [9]:
#NICE